In [31]:
import pandas as pd
import numpy as np
import os
import json
import csv
import requests
import prepare
import unicodedata
import string
from env import github_token, github_username
import time
import random
from bs4 import BeautifulSoup
from requests import get
import acquire
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from helpers import nlp
from sklearn.model_selection import train_test_split

#Modeling imports
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [2]:
# Bringing in the data using acquire function
df = acquire.wrangle_data()

In [3]:
#Checking to see how many repos we have
df.shape

(1031, 3)

In [4]:
# Getting a glimpse of the data
df.head()

,repo,language,readme_contents
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner](https://s3.amaz...
1,996icu/996.ICU,NaN,[996.ICU](https://996.icu/#/en_US)\n=======\n*...
2,EbookFoundation/free-programming-books,NaN,# List of Free Learning Resources In Many Lang...
3,jwasham/coding-interview-university,NaN,# Coding Interview University\n\n> I originall...
4,kamranahmedse/developer-roadmap,TypeScript,"<p align=""center"">\n <img src=""public/brand.p..."


In [5]:
# Converting readme to a string to run through the prepare functions
df['readme_contents'] =  df['readme_contents'].astype('string')

In [6]:
#Filling the null values in readme contents with an empty space
df.readme_contents = df.readme_contents.fillna('')

In [7]:
#Checking the total sums
df.isnull().sum()

repo                 0
language           114
readme_contents      0
dtype: int64

In [8]:
# Remove markdown links and html code
links = re.compile(r'(\(http.+\))')
html = re.compile(r'(<.*?>)')
df.readme_contents = df.readme_contents.replace(html, '')
df.readme_contents = df.readme_contents.replace(links, '')

In [9]:
#Checking data
df.head()

,repo,language,readme_contents
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner] [![Pull Reques...
1,996icu/996.ICU,NaN,[996.ICU] ======= **Please note that there exi...
2,EbookFoundation/free-programming-books,NaN,# List of Free Learning Resources In Many Lang...
3,jwasham/coding-interview-university,NaN,# Coding Interview University > I originally ...
4,kamranahmedse/developer-roadmap,TypeScript,"roadmap.sh Community driven roadmaps, ..."


In [10]:
#Applying the cleaning function to the dataframe to normalize text
df['cleaned_readme'] = df['readme_contents'].map(lambda x: prepare.basic_clean(x))

In [11]:
#Checking the data
df.head()

,repo,language,readme_contents,cleaned_readme
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner] [![Pull Reques...,freecodecamp social requests source freec...
1,996icu/996.ICU,NaN,[996.ICU] ======= **Please note that there exi...,note that there exists no other official acco...
2,EbookFoundation/free-programming-books,NaN,# List of Free Learning Resources In Many Lang...,list of free learning resources in many cc b...
3,jwasham/coding-interview-university,NaN,# Coding Interview University > I originally ...,coding interview i originally created this a...
4,kamranahmedse/developer-roadmap,TypeScript,"roadmap.sh Community driven roadmaps, ...",community driven roadmaps articles and r...


In [12]:
# Tokenizing the readme contents from prepare.py
df['tokenized_readme'] = df['cleaned_readme'].map(lambda x: prepare.tokenize(x))

In [13]:
#Checking the data to see changes
df.head()

,repo,language,readme_contents,cleaned_readme,tokenized_readme
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner] [![Pull Reques...,freecodecamp social requests source freec...,freecodecamp social requests source freecodeca...
1,996icu/996.ICU,NaN,[996.ICU] ======= **Please note that there exi...,note that there exists no other official acco...,note that there exists no other official accou...
2,EbookFoundation/free-programming-books,NaN,# List of Free Learning Resources In Many Lang...,list of free learning resources in many cc b...,list of free learning resources in many cc by ...
3,jwasham/coding-interview-university,NaN,# Coding Interview University > I originally ...,coding interview i originally created this a...,coding interview i originally created this as ...
4,kamranahmedse/developer-roadmap,TypeScript,"roadmap.sh Community driven roadmaps, ...",community driven roadmaps articles and r...,community driven roadmaps articles and resourc...


In [14]:
# Lemmatizing the data using preapre.py
df['lemm_readme'] = df['tokenized_readme'].map(lambda x: prepare.lemmatize(x))

In [15]:
#checking the data
df.head()

,repo,language,readme_contents,cleaned_readme,tokenized_readme,lemm_readme
0,freeCodeCamp/freeCodeCamp,TypeScript,[![freeCodeCamp Social Banner] [![Pull Reques...,freecodecamp social requests source freec...,freecodecamp social requests source freecodeca...,freecodecamp social request source freecodecam...
1,996icu/996.ICU,NaN,[996.ICU] ======= **Please note that there exi...,note that there exists no other official acco...,note that there exists no other official accou...,note that there exists no other official accou...
2,EbookFoundation/free-programming-books,NaN,# List of Free Learning Resources In Many Lang...,list of free learning resources in many cc b...,list of free learning resources in many cc by ...,list of free learning resource in many cc by t...
3,jwasham/coding-interview-university,NaN,# Coding Interview University > I originally ...,coding interview i originally created this a...,coding interview i originally created this as ...,coding interview i originally created this a a...
4,kamranahmedse/developer-roadmap,TypeScript,"roadmap.sh Community driven roadmaps, ...",community driven roadmaps articles and r...,community driven roadmaps articles and resourc...,community driven roadmaps article and resource...


In [16]:
# determining remaining list of languages that show up more than 10 times
languages_kept = df.language.value_counts()[df.language.value_counts() > 10].index
languages_kept

Index(['JavaScript', 'Python', 'TypeScript', 'Go', 'Java', 'C++', 'C', 'HTML',
       'Shell', 'Rust', 'Swift', 'Ruby', 'PHP', 'Jupyter Notebook', 'C#',
       'CSS', 'Kotlin'],
      dtype='object')

In [17]:
# Removing languages not kept
df = df[df.language.isin(languages_kept)]

In [18]:
#Checking to see how many rows we have left
df.shape, df.dtypes

((857, 6),
 repo                object
 language            object
 readme_contents     string
 cleaned_readme      object
 tokenized_readme    object
 lemm_readme         object
 dtype: object)

In [19]:
#No language nulls to be filled
df.language.isnull().value_counts()

False    857
Name: language, dtype: int64

# Modeling
---
### During the process we worked with processing data in different ways. Please see josh's working notebook for how we prepared and modeled data differently than in our final notebook. 
Please use the below url to see Josh's notebook:
https://github.com/david-jarad-josh-saroj/project-github-readme-classification/blob/master/josh-workbook.ipynb

In [20]:
#Splitting the data to ensure not data leakage
train, validate, test = prepare.split_lang(df)

In [21]:
#Checking shapes
train.shape, validate.shape, test.shape

((548, 6), (137, 6), (172, 6))

In [22]:
#Creating a baseline
baseline = 149/548
baseline

0.2718978102189781

In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer
#Vectorizing the text for model use
tfidf = TfidfVectorizer()
tfidf.fit(train.readme_contents)
# tfidf.fit(df.lemma)

xt = tfidf.transform(train.readme_contents).toarray()
xs = tfidf.transform(test.readme_contents).toarray()
xv = tfidf.transform(validate.readme_contents).toarray()

In [24]:
# Setting the Y variable
yt = train.language
ys = test.language
yv = validate.language

In [26]:
# Encoding the type of failure to pass thru XGBoost
le = LabelEncoder()
yt_encoded = le.fit_transform(yt)
yv_encoded = le.fit_transform(yv)

In [27]:

# fit model on training data and making classification report
model = XGBClassifier()
    
model.fit(xt, yt_encoded)

y_pred = model.predict(xt)

y_v_pred = model.predict(xv)
 
print(classification_report(yt_encoded, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        19
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        36
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00        50
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00        45
           7       1.00      1.00      1.00       149
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00         9
          11       0.99      1.00      0.99        73
          12       1.00      1.00      1.00        11
          13       1.00      1.00      1.00        14
          14       1.00      1.00      1.00        14
          15       1.00      1.00      1.00        11
          16       1.00      1.00      1.00        70

    accuracy              

In [28]:
#Checking how well it did on validate
print(classification_report(yv_encoded, y_v_pred))

              precision    recall  f1-score   support

           0       0.67      0.40      0.50         5
           1       0.00      0.00      0.00         2
           2       0.44      0.44      0.44         9
           3       0.00      0.00      0.00         2
           4       0.53      0.75      0.62        12
           5       1.00      0.25      0.40         4
           6       0.64      0.82      0.72        11
           7       0.60      0.86      0.71        37
           8       0.00      0.00      0.00         2
           9       0.50      0.50      0.50         2
          10       1.00      1.00      1.00         2
          11       0.73      0.61      0.67        18
          12       0.67      0.67      0.67         3
          13       1.00      0.67      0.80         3
          14       0.67      0.50      0.57         4
          15       1.00      0.67      0.80         3
          16       0.60      0.33      0.43        18

    accuracy              

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:

# fit model no training data and classification report
model2 = XGBRFClassifier()
    
model2.fit(xt, yt_encoded)

y_pred2 = model2.predict(xt)

y_v_pred2 = model2.predict(xv)
 
print(classification_report(yt_encoded, y_pred2))

              precision    recall  f1-score   support

           0       0.88      0.74      0.80        19
           1       0.71      0.62      0.67         8
           2       0.90      0.75      0.82        36
           3       1.00      0.25      0.40         8
           4       0.89      1.00      0.94        50
           5       1.00      0.29      0.44        14
           6       0.84      0.93      0.88        45
           7       0.85      0.95      0.90       149
           8       1.00      0.33      0.50         9
           9       0.80      0.50      0.62         8
          10       0.80      0.89      0.84         9
          11       0.78      0.93      0.85        73
          12       1.00      0.55      0.71        11
          13       1.00      1.00      1.00        14
          14       0.91      0.71      0.80        14
          15       0.85      1.00      0.92        11
          16       0.94      0.89      0.91        70

    accuracy              

In [30]:
#Checking to see how well it did on validate
print(classification_report(yv_encoded, y_v_pred2))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.00      0.00      0.00         2
           2       0.50      0.11      0.18         9
           3       0.00      0.00      0.00         2
           4       0.56      0.75      0.64        12
           5       0.00      0.00      0.00         4
           6       0.64      0.82      0.72        11
           7       0.54      0.81      0.65        37
           8       0.00      0.00      0.00         2
           9       0.50      0.50      0.50         2
          10       0.67      1.00      0.80         2
          11       0.71      0.67      0.69        18
          12       0.00      0.00      0.00         3
          13       0.67      0.67      0.67         3
          14       1.00      0.25      0.40         4
          15       0.50      0.67      0.57         3
          16       0.58      0.39      0.47        18

    accuracy              

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
#Setting SVM linear model to train and looking at the report
svm_model_linear = SVC(kernel = 'linear', C = 10).fit(xt, yt_encoded)
svm_predictions = svm_model_linear.predict(xv)
  
    # model accuracy for X_test  
accuracy = svm_model_linear.score(xv, yv_encoded)
  
    # creating a confusion matrix
#cm = confusion_matrix(y_validate, svm_predictions)

y_pred = svm_model_linear.predict(xt)

y_v_pred = svm_model_linear.predict(xv)

 
print(classification_report(yt_encoded, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        19
           1       1.00      1.00      1.00         8
           2       1.00      0.97      0.99        36
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00        50
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00        45
           7       0.99      1.00      1.00       149
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00         9
          11       0.99      1.00      0.99        73
          12       1.00      1.00      1.00        11
          13       1.00      1.00      1.00        14
          14       1.00      1.00      1.00        14
          15       1.00      1.00      1.00        11
          16       1.00      1.00      1.00        70

    accuracy              

In [33]:
#Looking at the report on validate
print(classification_report(yv_encoded, y_v_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         2
           2       0.25      0.11      0.15         9
           3       0.00      0.00      0.00         2
           4       0.83      0.42      0.56        12
           5       0.00      0.00      0.00         4
           6       0.62      0.45      0.53        11
           7       0.35      0.89      0.50        37
           8       1.00      0.50      0.67         2
           9       1.00      0.50      0.67         2
          10       0.00      0.00      0.00         2
          11       0.77      0.56      0.65        18
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         3
          14       1.00      0.25      0.40         4
          15       1.00      0.33      0.50         3
          16       0.43      0.17      0.24        18

    accuracy              

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#
---
Running the cleaned and lemmatized data through the models

In [34]:
# Running the vectorization on the lemma version of 
tfidf_lem = TfidfVectorizer()
tfidf_lem.fit(train.lemm_readme)
# tfidf.fit(df.lemma)

xtl = tfidf_lem.transform(train.lemm_readme).toarray()
xsl = tfidf_lem.transform(test.lemm_readme).toarray()
xvl = tfidf_lem.transform(validate.lemm_readme).toarray()

In [36]:
# fit model no training data
model = XGBClassifier()
    
model.fit(xtl, yt_encoded)

y_pred = model.predict(xtl)

y_v_pred = model.predict(xvl)
 
print(classification_report(yt_encoded, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        19
           1       1.00      1.00      1.00         8
           2       1.00      0.97      0.99        36
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00        50
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00        45
           7       0.97      1.00      0.99       149
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00         9
          11       1.00      0.97      0.99        73
          12       1.00      1.00      1.00        11
          13       1.00      1.00      1.00        14
          14       1.00      1.00      1.00        14
          15       1.00      1.00      1.00        11
          16       1.00      1.00      1.00        70

    accuracy              

In [37]:
print(classification_report(yv_encoded, y_v_pred))

              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.50      0.50      0.50         2
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00         2
           4       0.50      0.75      0.60        12
           5       0.00      0.00      0.00         4
           6       0.50      0.55      0.52        11
           7       0.50      0.84      0.63        37
           8       1.00      0.50      0.67         2
           9       0.00      0.00      0.00         2
          10       1.00      1.00      1.00         2
          11       0.77      0.56      0.65        18
          12       0.33      0.33      0.33         3
          13       0.50      0.33      0.40         3
          14       0.50      0.25      0.33         4
          15       1.00      0.67      0.80         3
          16       0.64      0.39      0.48        18

    accuracy              

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
